<a href="https://colab.research.google.com/github/123Vind/Diabetis-Predictor/blob/master/diabetisNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks


In [97]:
#importing dataset from google drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
diabetis =  pd.read_csv('/content/drive/My Drive/Colab Notebooks/diabetis.csv',header = None)

In [99]:
diabetis.head

<bound method NDFrame.head of       0    1   2   3    4     5      6   7  8
0     6  148  72  35    0  33.6  0.627  50  1
1     1   85  66  29    0  26.6  0.351  31  0
2     8  183  64   0    0  23.3  0.672  32  1
3     1   89  66  23   94  28.1  0.167  21  0
4     0  137  40  35  168  43.1  2.288  33  1
..   ..  ...  ..  ..  ...   ...    ...  .. ..
763  10  101  76  48  180  32.9  0.171  63  0
764   2  122  70  27    0  36.8  0.340  27  0
765   5  121  72  23  112  26.2  0.245  30  0
766   1  126  60   0    0  30.1  0.349  47  1
767   1   93  70  31    0  30.4  0.315  23  0

[768 rows x 9 columns]>

In [100]:
X = diabetis.iloc[:,:-1]
y = diabetis.iloc[:,-1]


In [101]:
#scaling the training data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


In [102]:
X

,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [103]:

y.value_counts()


0    500
1    268
Name: 8, dtype: int64

In [104]:
#to get rid of the unbalanced class
oversample = SMOTE()
X_res, y_res = oversample.fit_resample(X_scaled, y)
#tl = TomekLinks(sampling_strategy=0.5)
#X_res, y_res = tl.fit_resample(X,y)

In [105]:
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,random_state = 2,test_size = 0.20,stratify = y_res)

In [106]:
X_train

array([[0.23529412, 0.75879397, 0.73770492, ..., 0.44262295, 0.09222886,
        0.25      ],
       [0.17647059, 0.81909548, 0.57377049, ..., 0.4709389 , 0.08112724,
        0.11666667],
       [0.23529412, 0.72864322, 0.67213115, ..., 0.48435171, 0.06703672,
        0.81666667],
       ...,
       [0.11764706, 0.50753769, 0.47540984, ..., 0.32488823, 0.03287788,
        0.01666667],
       [0.29411765, 0.49748744, 0.60655738, ..., 0.43219076, 0.05337319,
        0.18333333],
       [0.41176471, 0.92462312, 0.68852459, ..., 0.5290611 , 0.11827498,
        0.33333333]])

In [107]:
y_res.value_counts()

1    500
0    500
Name: 8, dtype: int64

In [108]:
y_train.shape

(800,)

In [109]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = 2

In [110]:
X_train.shape

(800, 8)

In [111]:
def build_model():
  model = Sequential()
  model.add(Dense(6,input_dim=8,activation='relu'))
  model.add(Dense(8,activation='relu'))
  model.add(Dense(12,activation='relu'))
  model.add(Dense(10,activation='relu'))
  model.add(Dense(num_classes,activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  return model

In [112]:
model = build_model()

In [113]:
y_train.shape

(800, 2)

In [114]:
y_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [115]:
model.summary()
sample_weight = np.ones(shape=(len(y_train),2))
#sample_weight.shape
sample_weight[y_train==[0.,1.] ] = 0.5

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 6)                 54        
                                                                 
 dense_16 (Dense)            (None, 8)                 56        
                                                                 
 dense_17 (Dense)            (None, 12)                108       
                                                                 
 dense_18 (Dense)            (None, 10)                130       
                                                                 
 dense_19 (Dense)            (None, 2)                 22        
                                                                 
Total params: 370
Trainable params: 370
Non-trainable params: 0
_________________________________________________________________


In [116]:
model.fit(X_train,y_train,epochs=500,verbose=2,batch_size=32)

Epoch 1/500
25/25 - 1s - loss: 0.6875 - accuracy: 0.6037 - 735ms/epoch - 29ms/step
Epoch 2/500
25/25 - 0s - loss: 0.6790 - accuracy: 0.6800 - 38ms/epoch - 2ms/step
Epoch 3/500
25/25 - 0s - loss: 0.6703 - accuracy: 0.6787 - 40ms/epoch - 2ms/step
Epoch 4/500
25/25 - 0s - loss: 0.6597 - accuracy: 0.6862 - 37ms/epoch - 1ms/step
Epoch 5/500
25/25 - 0s - loss: 0.6441 - accuracy: 0.6800 - 45ms/epoch - 2ms/step
Epoch 6/500
25/25 - 0s - loss: 0.6244 - accuracy: 0.6837 - 54ms/epoch - 2ms/step
Epoch 7/500
25/25 - 0s - loss: 0.6045 - accuracy: 0.6812 - 49ms/epoch - 2ms/step
Epoch 8/500
25/25 - 0s - loss: 0.5873 - accuracy: 0.7100 - 43ms/epoch - 2ms/step
Epoch 9/500
25/25 - 0s - loss: 0.5761 - accuracy: 0.7038 - 42ms/epoch - 2ms/step
Epoch 10/500
25/25 - 0s - loss: 0.5673 - accuracy: 0.7125 - 39ms/epoch - 2ms/step
Epoch 11/500
25/25 - 0s - loss: 0.5589 - accuracy: 0.7188 - 72ms/epoch - 3ms/step
Epoch 12/500
25/25 - 0s - loss: 0.5500 - accuracy: 0.7188 - 40ms/epoch - 2ms/step
Epoch 13/500
25/25 - 0s

In [117]:
y_pred = model.predict(X_test)

In [118]:
y_pred

array([[0.60724795, 0.6540582 ],
       [0.8517494 , 0.43883947],
       [0.9912836 , 0.09176993],
       [0.84197384, 0.47727558],
       [0.8698523 , 0.4294077 ],
       [0.69650227, 0.5809133 ],
       [0.978649  , 0.15440422],
       [0.39976084, 0.7837025 ],
       [0.53139   , 0.7203197 ],
       [0.66892815, 0.71875393],
       [0.7276112 , 0.5850566 ],
       [0.44128555, 0.7919018 ],
       [0.98379046, 0.12595248],
       [0.5968268 , 0.7100804 ],
       [0.9940312 , 0.0776431 ],
       [0.60501814, 0.7210741 ],
       [0.4831604 , 0.76610565],
       [0.6185025 , 0.6753161 ],
       [0.64569056, 0.663756  ],
       [0.3094587 , 0.8389027 ],
       [0.4873097 , 0.74507564],
       [0.6156266 , 0.6818292 ],
       [0.25691187, 0.876709  ],
       [0.35779625, 0.8106152 ],
       [0.38406557, 0.80466974],
       [0.90977997, 0.36186367],
       [0.47454444, 0.77007127],
       [0.24002564, 0.86929554],
       [0.7424339 , 0.5384283 ],
       [0.44988027, 0.7649914 ],
       [0.

In [119]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [120]:
y_test = np.argmax(y_test,axis = 1)
y_predict = np.argmax(y_pred,axis=1)

In [121]:
y_test.shape

(200,)

In [122]:
y_predict

array([1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1])

In [123]:
confumat = metrics.confusion_matrix(y_test,y_predict)

In [124]:
confumat

array([[82, 18],
       [21, 79]])

In [125]:
result = metrics.classification_report(y_test,y_predict)
print()
print('Classification Results')
print(result)


Classification Results
              precision    recall  f1-score   support

           0       0.80      0.82      0.81       100
           1       0.81      0.79      0.80       100

    accuracy                           0.81       200
   macro avg       0.81      0.80      0.80       200
weighted avg       0.81      0.81      0.80       200

